In [98]:
import os
import shutil
# import random

# !pip install tqdm --upgrade
from tqdm.notebook import tqdm

In [99]:
# import ultralytics
# ultralytics.checks()

# Augment Test Image

In [ ]:
# %pip install -U albumentations

In [ ]:
#get image,label path

get_image_path = []
get_label_path = []

path = "/Users/boomctr/Documents/VisualCode/Senier_Project/data_1000pics/test"

labels = list(set([name for name in os.listdir(f"{path}/labels")]))
# print(labels)

for i in labels:
  a=i.replace('txt','jpg')
  get_image_path.append(f"{path}/images/{a}")
  get_label_path.append(f"{path}/labels/{i}")

print(get_image_path)
print(get_label_path)

In [ ]:
import cv2
import albumentations as A
from matplotlib import pyplot as plt
import os

# Function to read image dimensions
def get_image_dimensions(image_path):
    img = cv2.imread(image_path)
    height, width, _ = img.shape
    return width, height

# Function to read YOLO-formatted labels with classes
def read_yolo_file_with_class(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    bounding_boxes_with_classes = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) >= 5:
            class_label = int(parts[0])
            x_center = float(parts[1])
            y_center = float(parts[2])
            width = float(parts[3])
            height = float(parts[4])

            bounding_boxes_with_classes.append((class_label, x_center, y_center, width, height))
    # print(bounding_boxes_with_classes)
    return bounding_boxes_with_classes

# Function to augment images with YOLO-formatted bounding boxes and classes, and save them
def augment_and_save_images(image_paths, label_paths, save_path, num_augmentations=5):
    # Define augmentation pipeline using Albumentations
    augmentation_pipeline = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.RandomBrightnessContrast(p=0.2),
        A.Affine(shear=(-10, 10),p=0.5),
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

    # Load images and YOLO-formatted bounding boxes with classes
    for i, (image_path, label_path) in enumerate(zip(image_paths, label_paths)):
        # Get image dimensions dynamically
        image_width, image_height = get_image_dimensions(image_path)

        # Load image
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Load YOLO-formatted bounding boxes and classes from label file
        bounding_boxes_with_classes = read_yolo_file_with_class(label_path)

        # Convert YOLO coordinates to Albumentations format
        boxes_albumentations = [
            (box[1], box[2], box[3], box[4])
            for box in bounding_boxes_with_classes
        ]
        print(boxes_albumentations)
        class_labels = [box[0] for box in bounding_boxes_with_classes]

        # Augment images and bounding boxes
        for j in range(num_augmentations):
            augmented = augmentation_pipeline(image=image, bboxes=boxes_albumentations, class_labels=class_labels)
            augmented_image = augmented['image']
            augmented_boxes = augmented['bboxes']

            # Save augmented image
            os.makedirs(f"{save_path}/images/", exist_ok = True)
            save_filename = f"{save_path}/images/augmented_{i}_{j}.png"
            cv2.imwrite(save_filename, cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))

            # Save augmented bounding boxes with class IDs in YOLO format
            os.makedirs(f"{save_path}/labels/", exist_ok = True)
            label_filename = f"{save_path}/labels/augmented_{i}_{j}_label.txt"
            with open(label_filename, 'w') as label_file:
                for box, class_label in zip(augmented_boxes, class_labels):
                    x_center = box[0]
                    y_center = box[1]
                    width = box[2]
                    height = box[3]

                    label_file.write(f"{class_label} {x_center} {y_center} {width} {height}\n")

if os.path.exists("/content/augmented_images/"):
    print("Delete folder...")
    shutil.rmtree('/content/augmented_images/')
os.makedirs('./augmented_images/', exist_ok = True)

# Augment and save images with YOLO-formatted bounding boxes and classes
# augment_and_save_images(get_image_path, get_label_path, save_path="augmented_images", num_augmentations=5)
augment_and_save_images(['/Users/boomctr/Documents/VisualCode/Senier_Project/data_1000pics/test/images/IMG_3504_jpg.rf.1aafeb0baf95a03c0928fdd5eddc10e3.jpg'], ['/Users/boomctr/Documents/VisualCode/Senier_Project/data_1000pics/test/labels/IMG_3504_jpg.rf.1aafeb0baf95a03c0928fdd5eddc10e3.txt'], save_path="augmented_images", num_augmentations=5)

In [ ]:
#visualize image

import cv2
import albumentations as A

from matplotlib import pyplot as plt
import matplotlib.patches as patches
from matplotlib.image import imsave

def yolo_to_coco(yolo_annotation_path,image_width,image_height):

    bboxes = []
    category_ids = []

    with open(yolo_annotation_path, "r") as f:
        lines = f.readlines()

    for line in lines:
        parts = line.strip().split()
        class_index = int(parts[0])
        x_center, y_center, box_width, box_height = map(float, parts[1:5])

        # Convert YOLO coordinates to COCO format
        x_1 = int((x_center - box_width / 2) * image_width)
        y_1 = int((y_center - box_height / 2) * image_height)
        x_2 = int(box_width * image_width)
        y_2 = int(box_height * image_height)

        print([x_center, y_center, box_width, box_height])
        print([x_1,y_1,x_2,y_2])

        bboxes.append([x_1,y_1,x_2,y_2])
        category_ids.append(class_index)

    return bboxes, category_ids

#------------------------------------
BOX_COLOR = (255, 0, 0) # Red
TEXT_COLOR = (255, 255, 255) # White

def visualize_bbox(img, bbox, class_name, color=BOX_COLOR, thickness=4):

    x_min, y_min, w, h = bbox
    x_min, x_max, y_min, y_max = int(x_min), int(x_min + w), int(y_min), int(y_min + h)

    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)

    ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 1)
    cv2.rectangle(img, (x_min, y_min - int(1.6 * text_height)), (x_min + text_width, y_min), BOX_COLOR, -1)
    cv2.putText(
        img,
        text=class_name,
        org=(x_min, y_min - int(0.3 * text_height)),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=0.9,
        color=TEXT_COLOR,
        lineType=cv2.LINE_AA,
    )
    return img

def visualize(image, bboxes, category_ids, category_id_to_name):
    img = image.copy()
    for bbox, category_id in zip(bboxes, category_ids):
        class_name = category_id_to_name[category_id]
        img = visualize_bbox(img, bbox, class_name)
    plt.figure(figsize=(12, 12))
    plt.axis('off')
    plt.imshow(img)

#------------------------------------


def get_image_dimensions(image_path):
    img = cv2.imread(image_path)
    height, width, _ = img.shape
    return width, height

category_id_to_name = {
    0: 'Allergis',
    1: 'Black-Tiger',
    2: 'Black-Tiger_Big',
    3: 'Blackmores',
    4: 'Cisteine',
    5: 'Cisteine_Box',
    6: 'Grafadex',
    7: 'Heamovit',
    8: 'Iyara',
    9: 'Lac-Oph_Red',
    10: 'Ma-Kham-Pom',
    11: 'Ma-Kham-Pom_Bottle',
    12: 'Ma-Vaeng_Light',
    13: 'Ma-Vaeng_Syrup',
    14: 'Muclear',
    15: 'Mucolax',
    16: 'Nac-Long',
    17: 'NatC',
    18: 'Nevramin',
    19: 'Newpharvit',
    20: 'Opplin',
    21: 'Opsar',
    22: 'Pei-Pa-Koa',
    23: 'Pei-Pa-Koa_Big',
    24: 'Rhinalax',
    25: 'Rhinalax-B',
    26: 'Solmax',
    27: 'Solmax-Capsule',
    28: 'VapoRub_Big',
    29: 'VapoRub_Mid',
    30: 'VapoRub_Small',
    31: 'Vislube20',
    32: 'Vislube60'
}

# image_path = '/Users/boomctr/Documents/VisualCode/Senier_Project/data_1000pics/test/images/IMG_3504_jpg.rf.1aafeb0baf95a03c0928fdd5eddc10e3.jpg'
# label_path = '/Users/boomctr/Documents/VisualCode/Senier_Project/data_1000pics/test/labels/IMG_3504_jpg.rf.1aafeb0baf95a03c0928fdd5eddc10e3.txt'

image_path = '/content/augmented_images/images/augmented_0_3.png'
label_path = image_path.replace(".png","_label.txt").replace("/images","/labels")

image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

width, height = get_image_dimensions(image_path)

bboxes, category_ids = yolo_to_coco(label_path,width,height)

visualize(image, bboxes, category_ids, category_id_to_name)

# Train Model

In [ ]:
import ultralytics
ultralytics.checks()

In [2]:
def train_yolo():
  # !yolo task=detect mode=train model=yolov8l.pt data=/Users/boomctr/Documents/VisualCode/Senier_Project/data/data.yaml epochs=500 imgsz=512 batch=8 project=/Users/boomctr/Documents/VisualCode/Senier_Project/training_results name=pharmacy500b8_512 patience=0 cache=True iou=0.7 augment=True degrees=25.0 fliplr=0.0 lr0=0.0001 optimizer='Adam' #300_512_8
  # !yolo task=detect mode=train model=yolov8l.pt data=/Users/boomctr/Documents/VisualCode/Senier_Project/data/data.yaml epochs=300 imgsz=512 batch=8 project=/Users/boomctr/Documents/VisualCode/Senier_Project/training_results name=pharmacy300b8_512SGD patience=0 cache=True iou=0.7 augment=True degrees=25.0 fliplr=0.0 lr0=0.001 optimizer='SGD'
  !yolo task=detect mode=train model=yolov8l.pt data=/Users/boomctr/Documents/VisualCode/Senier_Project/data/data.yaml epochs=300 imgsz=512 batch=8 project=/Users/boomctr/Documents/VisualCode/Senier_Project/training_results name=pharmacy300b8_512RMSProp patience=0 cache=True iou=0.7 augment=True degrees=25.0 fliplr=0.0 lr0=0.001 optimizer='RMSProp'

# train_yolo()

In [1]:
def train_yolo():
  # !yolo task=detect mode=train model=yolov8l.pt data=/Users/boomctr/Documents/VisualCode/Senier_Project/data_1000pics/data.yaml epochs=300 imgsz=640 batch=16 project=/Users/boomctr/Documents/VisualCode/Senier_Project/training_results name=pharmacy300b16_640Adam_NoAug patience=0 cache=True iou=0.7 fliplr=0.0 lr0=0.0001 optimizer='Adam'
  !yolo task=detect mode=train model=yolov8l.pt data=/Users/boomctr/Documents/VisualCode/Senier_Project/data_1000pics/data.yaml epochs=300 imgsz=640 batch=16 project=/Users/boomctr/Documents/VisualCode/Senier_Project/training_results name=pharmacy300b16_640SGD_NoAug patience=0 cache=True iou=0.7 fliplr=0.0 lr0=0.001 optimizer='SGD'

# train_yolo()

## Graph Train & Loss

In [ ]:
import csv
import pandas as pd
from matplotlib import pyplot as plt
from numpy import arange

# Load the training and validation loss dictionaries\
# df = pd.read_csv('/Users/boomctr/Documents/VisualCode/Senier_Project/training_results/pharmacy400b16/results.csv')
# print(df)

#pharmacy500b16
train_loss_pharmacy500b16 = pd.read_csv('/Users/boomctr/Documents/VisualCode/Senier_Project/training_results/pharmacy500b16/results.csv', usecols=['         train/cls_loss'])
val_loss_pharmacy500b16 = pd.read_csv('/Users/boomctr/Documents/VisualCode/Senier_Project/training_results/pharmacy500b16/results.csv', usecols=['           val/cls_loss'])

#pharmacy300b8_512SGD
train_loss_pharmacy300b8_512SGD = pd.read_csv('/Users/boomctr/Documents/VisualCode/Senier_Project/training_results/pharmacy300b8_512SGD/results.csv', usecols=['         train/cls_loss'])
val_loss_pharmacy300b8_512SGD = pd.read_csv('/Users/boomctr/Documents/VisualCode/Senier_Project/training_results/pharmacy300b8_512SGD/results.csv', usecols=['           val/cls_loss'])

#pharmacy300b16_640Adam_NoAug
train_loss_pharmacy300b8_512RMSProp = pd.read_csv('/Users/boomctr/Documents/VisualCode/Senier_Project/training_results/pharmacy300b8_512RMSProp/results.csv', usecols=['         train/cls_loss'])
val_loss_pharmacy300b8_512RMSProp = pd.read_csv('/Users/boomctr/Documents/VisualCode/Senier_Project/training_results/pharmacy300b8_512RMSProp/results.csv', usecols=['           val/cls_loss'])

# print(train_loss.values)

train_value_pharmacy300b8_512RMSProp = []
val_value_pharmacy300b8_512RMSProp = []
train_value_pharmacy500b16 = []
val_value_pharmacy500b16 = []
train_value_pharmacy300b8_512SGD = []
val_value_pharmacy300b8_512SGD = []


#pharmacy500b16
for value in train_loss_pharmacy500b16.values:
  train_value_pharmacy500b16.append(float(value[0]))

for value in val_loss_pharmacy500b16.values:
  val_value_pharmacy500b16.append(float(value[0]))

#pharmacy300b8_512SGD
for value in train_loss_pharmacy300b8_512SGD.values:
  train_value_pharmacy300b8_512SGD.append(float(value[0]))

for value in val_loss_pharmacy300b8_512SGD.values:
  val_value_pharmacy300b8_512SGD.append(float(value[0]))

#pharmacy300b8_512RMSProp
for value in train_loss_pharmacy300b8_512RMSProp.values:
  train_value_pharmacy300b8_512RMSProp.append(float(value[0]))

for value in val_loss_pharmacy300b8_512RMSProp.values:
  val_value_pharmacy300b8_512RMSProp.append(float(value[0]))

# print(train_value_pharmacy300b8_512SGD)
# print(val_value_pharmacy300b8_512SGD)

# Generate a sequence of integers to represent the epoch numbers
epochs = range(1, 301)
epochs_304 = range(1, 304)
epochs_195 = range(1, 195)

figure, axis = plt.subplots(3, 1)

axis[0].plot(epochs_304, train_value_pharmacy500b16, label='train/cls_loss')
axis[0].plot(epochs_304, val_value_pharmacy500b16, label='val/cls_loss')
axis[0].set_title('model 300b16_Adam_4000 lr0=0.0001 optimizer="Adam" 4000pics')

axis[1].plot(epochs, train_value_pharmacy300b8_512SGD, label='train/cls_loss')
axis[1].plot(epochs, val_value_pharmacy300b8_512SGD, label='val/cls_loss')
axis[1].set_title('model 300b16_SGD_4000 lr0=0.001 optimizer="SGD" 4000pics')

axis[2].plot(epochs_195, train_value_pharmacy300b8_512RMSProp, label='train/cls_loss')
axis[2].plot(epochs_195, val_value_pharmacy300b8_512RMSProp, label='val/cls_loss')
axis[2].set_title('model 300b16_RMSProp_4000 lr0=0.001 optimizer="RMSProp" 4000pics')

for ax in axis.flat:
  ax.set(xlabel='Epochs', ylabel='Loss')
  ax.set_xlim([0, 300])
  ax.set_ylim([0, 4])

for ax in axis.flat:
  ax.label_outer()

# Display the plot
plt.legend(loc='best')
plt.show()

print('Best optimizer is Adam.')

In [ ]:
import csv
import pandas as pd
from matplotlib import pyplot as plt
from numpy import arange

#pharmacy300b16_640Adam_NoAug
train_loss_pharmacy300b16_640Adam_NoAug = pd.read_csv('/Users/boomctr/Documents/VisualCode/Senier_Project/training_results/pharmacy300b16_640Adam_NoAug/results.csv', usecols=['         train/cls_loss'])
val_loss_pharmacy300b16_640Adam_NoAug = pd.read_csv('/Users/boomctr/Documents/VisualCode/Senier_Project/training_results/pharmacy300b16_640Adam_NoAug/results.csv', usecols=['           val/cls_loss'])

#pharmacy300b16_640SGD_NoAug
train_loss_pharmacy300b16_640SGD_NoAug = pd.read_csv('/Users/boomctr/Documents/VisualCode/Senier_Project/training_results/pharmacy300b16_640SGD_NoAug/results.csv', usecols=['         train/cls_loss'])
val_loss_pharmacy300b16_640SGD_NoAug = pd.read_csv('/Users/boomctr/Documents/VisualCode/Senier_Project/training_results/pharmacy300b16_640SGD_NoAug/results.csv', usecols=['           val/cls_loss'])


# print(train_loss.values)

train_value_pharmacy300b16_640Adam_NoAug = []
val_value_pharmacy300b16_640Adam_NoAug = []
train_value_pharmacy300b16_640SGD_NoAug = []
val_value_pharmacy300b16_640SGD_NoAug = []

#pharmacy300b16_640Adam_NoAug
for value in train_loss_pharmacy300b16_640Adam_NoAug.values:
  train_value_pharmacy300b16_640Adam_NoAug.append(float(value[0]))

for value in val_loss_pharmacy300b16_640Adam_NoAug.values:
  val_value_pharmacy300b16_640Adam_NoAug.append(float(value[0]))

#pharmacy300b16_640SGD_NoAug
for value in train_loss_pharmacy300b16_640SGD_NoAug.values:
  train_value_pharmacy300b16_640SGD_NoAug.append(float(value[0]))

for value in val_loss_pharmacy300b16_640SGD_NoAug.values:
  val_value_pharmacy300b16_640SGD_NoAug.append(float(value[0]))

# Generate a sequence of integers to represent the epoch numbers
epochs = range(1, 301)
epochs_304 = range(1, 304)


figure, axis = plt.subplots(2, 1)

axis[0].plot(epochs, train_value_pharmacy300b16_640Adam_NoAug, label='train/cls_loss')
axis[0].plot(epochs, val_value_pharmacy300b16_640Adam_NoAug, label='val/cls_loss')
axis[0].set_title('model 300b16_Adam_1000 lr0=0.0001 optimizer="Adam" 1000pics')

axis[1].plot(epochs, train_value_pharmacy300b16_640SGD_NoAug, label='train/cls_loss')
axis[1].plot(epochs, val_value_pharmacy300b16_640SGD_NoAug, label='val/cls_loss')
axis[1].set_title('model 300b16_SGD_1000 lr0=0.001 optimizer="SGD" 1000pics')

for ax in axis.flat:
  ax.set(xlabel='Epochs', ylabel='Loss')
  ax.set_xlim([0, 300])
  ax.set_ylim([0, 2])

for ax in axis.flat:
  ax.label_outer()

# Display the plot
plt.legend(loc='best')
plt.show()

In [ ]:
import csv
import pandas as pd
from matplotlib import pyplot as plt
from numpy import arange

#pharmacy300b16_640Adam_NoAug
map50_pharmacy300b16_640Adam_NoAug = pd.read_csv('/Users/boomctr/Documents/VisualCode/Senier_Project/training_results/pharmacy300b16_640Adam_NoAug/results.csv',usecols=['       metrics/mAP50(B)'])

map50_value_pharmacy300b16_640Adam_NoAug = []

#pharmacy300b16_640Adam_NoAug
for value in map50_pharmacy300b16_640Adam_NoAug.values:
  map50_value_pharmacy300b16_640Adam_NoAug.append(float(value[0]))

# Generate a sequence of integers to represent the epoch numbers
epochs = range(1, 301)


# Plot and label the training and validation loss values
plt.plot(epochs, map50_value_pharmacy300b16_640Adam_NoAug, label='metrics/mAP50')

# Add in a title and axes labels
plt.title('mAP50 and epochs')
plt.xlabel('Epochs')
plt.ylabel('mAP50')

# Display the plot
plt.legend(loc='best')
plt.show()

# Predict Images

In [101]:
def detect_img(class_name):
  a = []

  if class_name:
    for i in class_name.strip().split(','):
      if i == '':
        continue
      a.append(i)

    class_name=str(','.join(a))
    # print(1)
    !/opt/homebrew/anaconda3/envs/final/bin/yolo task=detect mode=predict model="/Users/boomctr/Documents/VisualCode/Senier_Project/training_results/pharmacy300b16_640Adam/weights/best.pt" conf=0.3 source="/Users/boomctr/Documents/VisualCode/Senier_Project/test.jpg" save=True save_txt=True line_width=1 project="/Users/boomctr/Documents/VisualCode/Senier_Project/detect_img" classes={class_name}
  else:
    # print(2)
    !/opt/homebrew/anaconda3/envs/final/bin/yolo task=detect mode=predict model="/Users/boomctr/Documents/VisualCode/Senier_Project/training_results/pharmacy300b16_640Adam/weights/best.pt" conf=0.3 source="/Users/boomctr/Documents/VisualCode/Senier_Project/test.jpg" save=True save_txt=True line_width=1  project="/Users/boomctr/Documents/VisualCode/Senier_Project/detect_img"

# detect_img("1,2,,3")

# Send Result

In [102]:
names = {
    0:"Allergis",
    1:"Black-Tiger",
    2:"Black-Tiger_Big",
    3:"Blackmores",
    4:"Cisteine",
    5:"Cisteine_Box",
    6:"Grafadex",
    7:"Heamovit",
    8:"Iyara",
    9:"Lac-Oph_Red",
    10:"Ma-Kham-Pom",
    11:"Ma-Kham-Pom_Bottle",
    12:"Ma-Vaeng_Light",
    13:"Ma-Vaeng_Syrup",
    14:"Muclear",
    15:"Mucolax",
    16:"Nac-Long",
    17:"NatC",
    18:"Nevramin",
    19:"Newpharvit",
    20:"Opplin",
    21:"Opsar",
    22:"Pei-Pa-Koa",
    23:"Pei-Pa-Koa_Big",
    24:"Rhinalax",
    25:"Rhinalax-B",
    26:"Solmax",
    27:"Solmax-Capsule",
    28:"VapoRub_Big",
    29:"VapoRub_Mid",
    30:"VapoRub_Small",
    31:"Vislube20",
    32:"Vislube60",
  }

names_list = [
    "Allergis",
    "Black-Tiger",
    "Black-Tiger_Big",
    "Blackmores",
    "Cisteine",
    "Cisteine_Box",
    "Grafadex",
    "Heamovit",
    "Iyara",
    "Lac-Oph_Red",
    "Ma-Kham-Pom",
    "Ma-Kham-Pom_Bottle",
    "Ma-Vaeng_Light",
    "Ma-Vaeng_Syrup",
    "Muclear",
    "Mucolax",
    "Nac-Long",
    "NatC",
    "Nevramin",
    "Newpharvit",
    "Opplin",
    "Opsar",
    "Pei-Pa-Koa",
    "Pei-Pa-Koa_Big",
    "Rhinalax",
    "Rhinalax-B",
    "Solmax",
    "Solmax-Capsule",
    "VapoRub_Big",
    "VapoRub_Mid",
    "VapoRub_Small",
    "Vislube20",
    "Vislube60",
  ]

## Capture Image From Webcam

In [103]:
# capture single image from webcam
import cv2 as cv
import time

def CapCam():
	# initialize the camera 
	cam_port = 0
	cam = cv.VideoCapture(cam_port)

	# reading the input using the camera 
	time.sleep(1)
	result, image = cam.read() 

	print("capture image...")

	# If image will detected without any error,show result 
	if result: 

		# saving image in local storage 
		cv.imwrite("/Users/boomctr/Documents/VisualCode/Senier_Project/test.jpg", image) 
		
		print("Image save to test.jpg.")

		return

	# If captured image is corrupted
	else: 
		print("No image detected. Please! try again") 

# CapCam()

## Message Algorithm

In [104]:
import time
from collections import Counter

def read_file(names,is_filter,filter_names = []):
  results = []
  ans = []
  all_class_detect=[]
  all_class_filter=[]

  time.sleep(1)

  #append class filter name into list
  if is_filter:
    for i in filter_names.strip().split(','):
      if i == '':
        continue
      if int(i) in names:
        if names[int(i)] not in all_class_filter:
          all_class_filter.append(names[int(i)])
  
  # print(all_class_filter)

  # check file is exist
  if os.path.exists('/Users/boomctr/Documents/VisualCode/Senier_Project/detect_img/predict/labels/test.txt'): #predict1

    with open(f'/Users/boomctr/Documents/VisualCode/Senier_Project/detect_img/predict/labels/test.txt') as f: #predict1
        lines = f.readlines()
        lines=[x.split(" ") for x in lines]
        # print(lines)
        for i in lines:
          results.append(i[0])
    # print(results)
    
    # count class name
    count_value = Counter(results)
    # print(count_value,len(count_value))

    # append class name into ans
    for value, count in count_value.items():
      # print(value, count)
      all_class_detect.append(names[int(value)])
      # print(all_class_detect)
      if count <= 1:
        ans.append(names[int(value)] + ' remain(s) ' + str(count) + '\n')
    
    for i in all_class_filter if is_filter else names_list :
      if i not in all_class_detect:
        ans.append(i + ' remain(s) 0' + '\n')
      
    # count > 1
    if len(ans) == 0:
      ans.append('No small amount of medicine' + '\n')

    # print(ans)
    if is_filter:
      message = 'Hello Boom' + '\n' + 'Filter: ' + ','.join(all_class_filter) + '\n' + ''.join(ans)
    else:
      message = 'Hello Boom' + '\n' + ''.join(ans)

    # print(ans)
    return message
  
  else:

    if is_filter:
      message = 'Hello Boom' + '\n' + 'Filter: ' + ','.join(all_class_filter) + '\n' + 'No medicine detected' + '\n' 
    else:
      message = 'Hello Boom' + '\n' + 'No medicine detected' + '\n' 
    return message

# print(read_file(names,is_filter=True,filter_names='1,2,,3'))

## Other Algorithm

In [105]:
# check image size
def check_img_size():
  res = os.stat('/Users/boomctr/Documents/VisualCode/Senier_Project/detect_img/predict/test.jpg').st_size
  print("file size (bytes)",res)
  return res

# check_img_size()

In [106]:
# high resolution image -> low resolution image
from PIL import Image

def low_resolution():
  print('low resolution image to below 3MB...')
  im = Image.open("/Users/boomctr/Documents/VisualCode/Senier_Project/detect_img/predict/test.jpg")
  im.save("/Users/boomctr/Documents/VisualCode/Senier_Project/detect_img/predict/test.jpg", dpi=(600,600))

# low_resolution()

## Line API Notify

In [107]:
import requests

def linenotify(message):
  url = 'https://notify-api.line.me/api/notify'
  token = 'xxxxxxxxxxxxxxxxxxxxxxxxxx' # Line Notify Token Y3Vtrey3Swmmuu8Ii8DkUCRPiGreLrOODDDzyVZlnWN
  data = {'message': message}
  headers = {'Authorization':'Bearer ' + token}
  session = requests.Session()

  # check image size < 3MB
  if check_img_size() > 3000000:
    print("need to low resolution")
    low_resolution()

  img = {'imageFile': open('/Users/boomctr/Documents/VisualCode/Senier_Project/detect_img/predict/test.jpg','rb')} #Local picture File
  session_post = session.post(url, headers=headers, data =data, files=img)

  print(session_post.text)


## Get Input For Filter Class

In [108]:
def get_valid_input():
  while True:
    user_input = input(f"Enter numbers and commas only: (separate with ,) {names}")

    if any(char.isdigit() or char == "," for char in user_input) and user_input.replace(",", "").isdigit() and not user_input.startswith(",") and not user_input.endswith(","):
      try:
        float(user_input.replace(",", ""))
        return user_input
      except ValueError:
        print("Invalid comma placement. Please use commas only between numbers.")
    else:
      print("Invalid input. Please enter only numbers (0-9) and commas.")



## Main Algorithm

In [109]:
from datetime import datetime
import pytz

filter_names=''

if os.path.exists("/Users/boomctr/Documents/VisualCode/Senier_Project/detect_img"):
  print("Delete folder...")
  shutil.rmtree('/Users/boomctr/Documents/VisualCode/Senier_Project/detect_img')
  print("Delete folder success")

tz_BK = pytz.timezone('Asia/Bangkok')
datetime_BK = datetime.now(tz_BK)

CapCam() #capture image

#filter class
#False = all class while True = filter class
is_filter = False #default
# is_filter = True

if is_filter:
  filter_names = get_valid_input()

  detect_img(filter_names)
else:
  detect_img('')

message = datetime_BK.strftime("%F %T.%f")[:-3] + "\n"
message = message + read_file(names,is_filter,filter_names)
linenotify(message)


Delete folder...
Delete folder success


Ultralytics YOLOv8.0.227 🚀 Python-3.9.18 torch-2.1.1 CPU (Apple M1)
Model summary (fused): 268 layers, 43632051 parameters, 0 gradients, 165.0 GFLOPs

image 1/1 /Users/boomctr/Documents/VisualCode/Senier_Project/test.jpg: 480x640 2 Black-Tigers, 1 Black-Tiger_Big, 5 Cisteines, 1 Cisteine_Box, 4 Heamovits, 4 Iyaras, 3 Lac-Oph_Reds, 5 Ma-Kham-Poms, 5 Ma-Kham-Pom_Bottles, 1 Ma-Vaeng_Light, 3 Ma-Vaeng_Syrups, 2 Muclears, 5 Mucolaxs, 4 Nac-Longs, 4 Nevramins, 4 Newpharvits, 6 Opplins, 6 Opsars, 3 Pei-Pa-Koas, 2 Pei-Pa-Koa_Bigs, 2 Rhinalaxs, 6 Rhinalax-Bs, 3 Solmax-Capsules, 1 VapoRub_Big, 1 VapoRub_Mid, 1 VapoRub_Small, 2 Vislube20s, 1 Vislube60, 471.8ms
Speed: 5.0ms preprocess, 471.8ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)
Results saved to /Users/boomctr/Documents/VisualCode/Senier_Project/detect_img/predict
1 label saved to /Users/boomctr/Documents/VisualCode/Senier_Project/detect_img/predict/labels
💡 Learn more at https://docs.ultralytics.com/modes/predict
file